# Análise de Negócio — Vendas Olist

Este notebook apresenta análises exploratórias sobre as vendas da Olist
utilizando o Data Warehouse construído no Databricks (camadas Silver e Gold).

As análises respondem perguntas-chave de negócio como:
- Evolução do faturamento ao longo do tempo
- Produtos e categorias mais relevantes
- Distribuição geográfica das vendas


In [ ]:
# Faturamento Total

spark.sql("""
SELECT
    SUM(valor_item) AS faturamento_total
FROM workspace.gold.fato_vendas
""").show()

In [ ]:
# Evolução mensal de Faturamento

df_faturamento_mes = spark.sql("""
SELECT
    d.ano,
    d.mes,
    SUM(f.valor_item) AS faturamento
FROM workspace.gold.fato_vendas f
JOIN workspace.silver.dim_data d
    ON f.id_data = d.id_data
GROUP BY d.ano, d.mes
ORDER BY d.ano, d.mes
""")

display(df_faturamento_mes)

In [ ]:
# Top 10 produtos por faturamento

df_top_produtos = spark.sql("""
SELECT
    f.id_produto,
    SUM(f.valor_item) AS faturamento
FROM workspace.gold.fato_vendas f
GROUP BY f.id_produto
ORDER BY faturamento DESC
LIMIT 10
""")

display(df_top_produtos)

In [ ]:
# Faturamento por categoria

df_categoria = spark.sql("""
SELECT
    p.nome_categoria,
    SUM(f.valor_item) AS faturamento
FROM workspace.gold.fato_vendas f
JOIN workspace.silver.dim_produto p
    ON f.id_produto = p.id_produto
GROUP BY p.nome_categoria
ORDER BY faturamento DESC
""")

display(df_categoria)

In [ ]:
#Faturamento por estado do cliente

df_estado = spark.sql("""
SELECT
    c.estado_cliente,
    SUM(f.valor_item) AS faturamento
FROM workspace.gold.fato_vendas f
JOIN workspace.silver.dim_cliente c
    ON f.id_cliente = c.id_cliente
GROUP BY c.estado_cliente
ORDER BY faturamento DESC
""")

display(df_estado)

## Conclusões

- O faturamento apresenta variação ao longo do tempo, indicando possíveis
  padrões de sazonalidade.
- Um pequeno conjunto de produtos concentra grande parte da receita.
- Algumas categorias se destacam significativamente no faturamento total.
- A distribuição geográfica mostra concentração de vendas em determinados estados.

Este MVP demonstra a viabilidade da construção de um Data Warehouse
utilizando Databricks e modelagem dimensional para suporte à decisão.
